In [4]:
import datetime
import time
import grequests
import aiohttp
import asyncio
import requests

TOKEN = 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJlbWFpbCI6IndlZ2V3b24yOTJAZ2V0bW9sYS5jb20iLCJ1cmxiYWNrIjoid3d3LmFsbHRpY2tldC5jb20iLCJwYXltZW50Q2hhbm5lbCI6IkMwNyIsInRpY2tldFR5cGUiOiIwMSIsImxhbmciOiJFIiwiZGF0YSI6Ijc5MzA2YWI1MjUzM2RmNzEzYTExMzU1NzdmODg4ZjY1YjA1NDBmOTIzMTU1NWI5N2MyMGYwMzU3YWZkODg3NTdiNTQ5NjliYzRmY2JiM2VhZTk0NzA5YTljNDAxYTk2M2I1M2Q2ZGM4ZmExZGI3NzA4ZjM0ZjhiYjI0YzZjZjYyOTNlMGQyYzdiNjU0YzZjNTc4ODk5NTgxOGFiOWYyZDY3MjgzNDU3ZmJhNjgxNTlhY2VhZjI5ZGNlYTJhNDc1MzBmMzdlMDJjMjVjMTQ0ZTQwNTgzMGMyZGFmNTVhNDc4ZjZhN2Q3NTQyMDgzYzljOTQzNWM3MTEzNjk3OWVhZmRhMzI2NjAzNTVkNjE4ZDQ4NjNhZGE5YjE2MmIwMzgxYTRlNDlmYTA0ODQ0ZGFmNDA3MWI4OTAyYTdiNjc4MWNlZWQ5OWEzNDEyOTIwZTI3Mjc2N2RlMTkwMzhlOGQ1ZTE0YTNkODI2Y2NjZTlhNmE4ZDdlZDdiNzhmZDc3ZDllZjE2NDc4MmIxODdiZWZhMTIxNGZjMmEwNmJkMDRmNzg1NGU3MDA5Mzc5ZmY3NzRlMWNkNDk0ZDIxYjhkMWZhNTEyMmFlMjRjMzZkYjU4YTlmM2QwMDFkYzY2YWNlNmVjNTQyYjQ3OGQ5NWFlZDRlY2Q1ZTQ4YjQ1MDkyMTZhNjQ1NTU5MGJmMWVlY2JlMWQ5NDU3YzIwNmY3YzE4MjM1MmM5ZDAwOTAzYmFiZjVkYzczNWVhNzI5Y2M1Zjg4NmJjODk3Y2Y5MzM5ZTBlNzEwNTJjNzQxNGRhNmM0NzdmZjJhMWZmMjE2NGQ2YzhhMjJlZTA0YzZmOWY4ZmU0ODZhOTAyYTkzZjExODIzZDcwOWY2M2M2MzQ5MWMxZjQ5MTU4YTdkNDQ5ZTI0MDFkOTlkODJkYjFmMDE1NWZhNTc0MDNiMThhY2U4ZWNlNzQzOWMxN2E2MGZhMGI2NzU1NzUwZmUiLCJ0aW1lU3RhbXAiOjAuMTg2MjI1ODUwODMzNjcwNzEsImlhdCI6MTcwMjE1MjEwNSwiZXhwIjoxNzAyMTYyOTA1LCJpc3MiOiJjc2F0azE4In0.XiXO_NKbGPEle8Y5RqfBIRKYInUsA7V8rvPtATWFS3Y'

class MenuManager:
    def __init__(self):
        self.base_data = self.get_base_data()
        self.concert_id = self.select_concert()
        
    def get_base_data(self):
        base_data = {
            'url': 'https://www.allticket.com/',
            'headers': {
                'authorization': TOKEN,
                'Content-Type': 'application/json'
            }
        }

        # req = base_data.copy()
        # req['url'] = 'https://api.allticket.com/customer/get-purchase-history'
        # req['json'] = { 'headers': { 'normalizedNames': {}, 'lazyUpdate': None } }
        
        # code = requests.post(**req).status_code
        # if code != 200:
        #     print("Wrong Token try again")
        #     return self.get_base_data()

        return base_data
        
    
    def select_concert(self) -> str:
        req = self.base_data.copy()
        req['url'] = 'https://api.allticket.com/content/get-all-events'
        req['json'] = { 'groupKey': 'concert' }
        res = requests.post(**req).json()
        items = res['data']['event']['items']
        
        for i, item in enumerate(items):
            print(f"ID: {i}")
            print(f"Name: {item['name']}")
            print(f"ชื่อ: {item['namePos']}")
            print()
        
        return items[self.get_user_input(convert_int=True)]['id']
        
    def get_user_input(self, message="Select Concert By ID: ", convert_int=False):
        user_i = input(message).strip()
        
        if convert_int:
            if not user_i.isnumeric():
                return self.get_user_input(message, convert_int)
            
            user_i = int(user_i)
        
        return user_i
    
    async def book_now(self):
        self.perform = await PerformManager(self.concert_id, self.base_data).book_seats()
        return self
    
    async def book_later(self):
        now = datetime.datetime.now()
        
        target_time = self.get_user_input("Please Input Time in this format %H:%M: ")
    
        target_hour, target_minute = map(int, target_time.split(':'))
        target_datetime = datetime.datetime(now.year, now.month, now.day, target_hour, target_minute)
        
        if now > target_datetime:
            target_datetime += datetime.timedelta(days=1)


        count_down_sec = 10
        notify_time = target_datetime - datetime.timedelta(seconds=count_down_sec)
        
        if now > notify_time:
            notify_time += datetime.timedelta(days=1)

        time_difference = notify_time - now
        total_seconds = time_difference.total_seconds()

        print()
        print(f"Booking Start At: {target_time}")
        print(f"In {total_seconds + count_down_sec} Seconds")
        print()
        time.sleep(total_seconds)

        for i in range(count_down_sec, 0, -1):
            print(f"Countdown: {i} seconds")
            time.sleep(1)

        print()
        print("Booking Start!")
        print()

        self.perform = await PerformManager(self.concert_id, self.base_data).book_seats()
        return self
    
    async def select_book_options(self):
        li = [
            self.book_now,
            self.book_later,
        ]

        print("[0] Book Now")
        print("[1] Schedule Book")
        
        user_i = self.get_user_input(message="Select Number: ", convert_int=True)

        return await li[user_i]()

class PerformManager:
    id: int
    round_id: int
    token: str
    event: str
    seat_needed: int
    zones: list
    booked_seat: list

    def __init__(self, id, base_data) -> None:
        self.id = id
        self.base_data = base_data
        self.seat_needed = None
        self.max_reserve_each = None
        self.round_id = None
        self.zone_greqs = []
        self.seat_greqs = []
        self.booked_seat = []
        self.zones = self.get_zones()
        self.seats = self.get_seats()
        
    def done(self):
        return self.seat_needed <= 0
    
    def get_base_data(self):
        return self.base_data.copy()

    def get_id(self):
        req = self.get_base_data()
        req['url'] = 'https://api.allticket.com/content/get-all-events'
        req['json'] = { 'groupKey': 'concert' }
        res = requests.post(**req).json()
        items = res['data']['event']['items']
        found_con = next(filter(lambda x: x['performUri'] == self.event, items), None)
        if not found_con:
            raise Exception('NOT FOUND CONCERT THIS NAME')

        self.id = found_con['id']

    def get_round_id(self):
        if not self.id:
            self.get_id()        

        req = self.get_base_data()
        req['url'] = 'https://api.allticket.com/booking/get-round'
        req['json'] = { 'performId': self.id }
        res = requests.post(**req).json()
        data = res['data']['event_info']
        self.max_reserve_each = data['maxReserve']
        self.seat_needed = data['maxSelectSeatPerUser']
        self.round_id = data['list_round'][0]['roundId']
        
    def get_zones(self):
        if not self.round_id:
            self.get_round_id()
        
        req = self.get_base_data()
        req['url'] = 'https://api.allticket.com/booking/seat-available'
        req['json'] = { 'performId': self.id, 'roundId': self.round_id }
        res = requests.post(**req).json()
        
        zones = []
        for zone in res['data']['seat_available']:
            if zone['amount'] <= self.seat_needed:
                continue
            
            zone_manager = ZoneManager(self, zone['id'])
            zones.append(zone_manager)
            self.zone_greqs.append(grequests.post(**zone_manager.req))

        return zones
    
    def get_seats(self):
        seat_needed = self.seat_needed

        for i, res in grequests.imap_enumerated(self.zone_greqs):
            zone = self.zones[i]
            data = res.json()['data']
        
            if not data:
                continue

            zone.zone_type = data['zone_type']
            seats = []
            
            for seat_data in data['seats_available']:
                screen_label = seat_data['screenLabel']
                
                seat_ids = []
                count = 0
                for seat in seat_data['seat']:
                    status = seat['status']
                    
                    if status != "A":
                        continue

                    seat_ids.append(f"{seat['rowName']}_{seat['seatNo']}")
    
                    count += 1
                    
                    if count >= seat_needed or count >= self.max_reserve_each:
                        seat_manager = SeatManager(zone, screen_label, seat_ids, count)
                        seats.append(seat_manager)
                        self.seat_greqs.append(grequests.post(**seat_manager.req))

                        count = 0
                        seat_ids = []

            return seats
    
    async def book_seats(self):
        payment_reqs = []

        for i, res in grequests.imap_enumerated(self.seat_greqs):
            seat = self.seats[i]
            response = res.json()
            data = response['data']
            success = response['success'] == True
            
            # if not success:
            #     continue

            seat.booking_id = data['uuid']
            seat.booking_wait_time = data['waitTime']
            seat.booking_time_out = data['timeOut']
            seat.booking_retry = data['retry']
            
            payment_reqs.append(seat.get_payment_req())

        payment_reqs = await asyncio.gather(*payment_reqs)
        
        for seat in payment_reqs:
            if not seat:
                continue
            
            req = self.get_base_data()
            req['url'] = 'https://api.allticket.com/payment/payment/outlet'
            req['json'] = {
                "payment_channel": "outlet",
                "performId": self.id,
                "reserveId": seat.reserve_id,
                "insureProducts": []
            }

            if self.done():
                break

            res = requests.post(**req).json()

            if res['success'] == True:
                self.seat_needed -= seat.seat_count
                self.booked_seat.append(seat)

        return self


class ZoneManager:
    id: str
    zone_type: str
    perform: PerformManager
    seat_reqs = []
    seats: list

    def __init__(self, perform, zone_id) -> None:
        self.perform = perform
        self.id = zone_id
        self.req = self.get_req()

    def done(self):
        self.perform.seat_needed <= 0
        
    def get_base_data(self):
        return self.perform.base_data.copy()
        
    def get_req(self):
        if self.done():
            return []
        
        req = self.get_base_data()
        perform = self.perform

        req['url'] = 'https://api.allticket.com/booking/get-seat'
        req['json'] = { 'performId': perform.id, 'roundId': perform.round_id, 'zoneId': self.id }
        
        return req

class SeatManager:
    screen_label: str
    ids: list
    zone: ZoneManager
    booking_id = None
    booking_wait_time = 5
    booking_time_out = 20
    booking_retry = 9
    reserve_id = None
    booked = False

    def __init__(self, zone, screen_label, seat_ids, seat_count) -> None:
        self.zone = zone
        self.screen_label = screen_label
        self.ids = seat_ids
        self.seat_count = seat_count
        self.req = self.get_req()

    def done(self):
        return self.zone.perform.seat_needed <= 0
    
    def get_base_data(self):
        return self.zone.perform.base_data.copy()
    
    def get_req(self):
        zone = self.zone
        perform = zone.perform

        req = self.get_base_data()
        req['url'] = 'https://api.allticket.com/booking/handler-reserve'
        req['json'] = {
            "performId": perform.id,
            "roundId": perform.round_id,
            "zoneId": zone.id,
            "screenLabel": self.screen_label,
            "seatTo": {
                "seatType": zone.zone_type,
                "seats": self.ids
            },
            "shirtTo": []
        }
        
        return req

    def book(self):
        res = requests.post(**self.req).json()
        data = res['data']
        self.booking_id = data['uuid']
        self.booking_wait_time = data['waitTime']
        self.booking_time_out = data['timeOut']
        self.booking_retry = data['retry']

    async def get_payment_req(self):
        req = self.get_base_data()
        req['url'] = 'https://api.allticket.com/booking/check-booking'
        req['json'] = { "uuid": self.booking_id }

        await asyncio.sleep(self.booking_wait_time)
        
        if self.done():
            return None

        async with aiohttp.ClientSession() as session:
            async with session.post(**req) as response:
                res = await response.json()
                success = res['success'] == True

                if success:
                    data = res['data']
                    self.reserve_id = data['reserveId']
                    return self

                return None


ID: 0
Name: 2024 RO WOON FANMEETING An Ordinary Day IN BANGKOK
ชื่อ: 2024 โร อุน แฟน มีตติ้ง อิน แบงค็อก

ID: 1
Name: Chang Music Connection presents เฉียงเหนือเฟส 2
ชื่อ: เฉียงเหนือเฟส 2

ID: 2
Name: YESUNG SOLO CONCERT - Unfading Sense in BANGKOK
ชื่อ: เยซอง โซโล่ คอนเสิร์ต - อันเฟดิง เซนส์ อิน แบงค็อก

ID: 3
Name: SEVENTEEN TOUR 'FOLLOW' TO BANGKOK
ชื่อ: เซเวนทีน ทัวร์ ฟอลโล ทู แบงค็อก

ID: 4
Name: ห้วยไร่ อีหลีน่า เฟสติวัล 2
ชื่อ: ห้วยไร่ อีหลีน่า เฟสติวัล 2

ID: 5
Name: คอนเสิร์ตการกุศล เพื่อชีวิต เพื่อกวนอู
ชื่อ: คอนเสิร์ตการกุศล เพื่อชีวิต เพื่อกวนอู

ID: 6
Name: Rimpha Music Festival 11
ชื่อ: ริมผา มหานิยม 11

ID: 7
Name: ROAD FOR LIFE EP 2
ชื่อ: โร้ดฟอร์ไลฟ์ อีพี สอง : วิถีเพื่อชีวิต

ID: 8
Name: Invictus Christmas Concert 2023
ชื่อ: งานคริสต์มาสคอนเสิร์ตของนานาชาติอินวิคตัส 2023

ID: 9
Name: Nova 75 Countdown Music Festival
ชื่อ: โนวา 75 เคาท์ดาวน์ มิวสิค เฟสติวัล

ID: 10
Name: Sek Loso Concert
ชื่อ: เสก โลโซ คอนเสิร์ต

ID: 11
Name: Black Sand Natural Song
ชื่อ: แบล็คแซนด์เนเ

AttributeError: 'MenuManager' object has no attribute 'token'